# Часть 1. Линейные методы. Регрессия.

В ноутбуке:
- Решим задачу линейной регрессии аналитически
- Решим задачу линейной регрессии численно, используя библиотеку sklearn

ссылка на ноутбук в колаб: https://colab.research.google.com/drive/1XF0JaI-klJF1KXzEICSrPXGpzQATnN9E?usp=sharing

## Немного теории

Линейные методы предполагают, что между признаками объекта и целевой переменной существует линейная зависимость, то есть:
$$ y = w_1 x_1 + w_2 x_2 + ... + w_k x_k + b $$,
где у - целевая переменная (что мы хотим предсказать), $x_i$ -- признак объекта х, $w_i$ -- вес i-го признака, b -- bias (смещение, свободный член)

Часто предполагают, что объект х содержит в себе фиктивный признак, который всегда равен 1, тогда bias это есть вес этого признака. В этом случае формула принимает простой вид:
$$ y = <w, x> $$,
где $<\cdot, \cdot>$ -- скалярное произведение векторов.

В матричной форме, в случае, когда у нас есть n объектов формулу можно переписать следующим образом:
$$ y = Xw $$,
y -- вектор размера n, X -- матрица объекты-признаки размера $n \times k$, w -- вектор весов размера k.

Решение по методу наименьших квадратов дает 
$$ w = (X^TX)^{-1}X^Ty $$

**Определение (Lp-норма):**

$$
    \|\cdot\|_{p}: \mathbb{R}^{d} \to \mathbb{R}\\
    \forall p \geq 1: \forall x \in \mathbb{R}^{d}: \|x\|_{p} = \sqrt[p]{\sum_{i=1}^{n} x_{i}^{p}}
$$

**Доказательство**

Вспомним, как выглядит задача оптимизации:

$$
    \frac{1}{n} \sum_{i=1}^{n} (y_i - \langle x_i, w \rangle)^2 \to \min\limits_{w}
$$

Эта задача оптимизации допускает следующую более удобную запись:

$$
    \frac{1}{n} \| Xw - y \|_{2}^{2} \to \min\limits_{w}
$$

Утверждается, что:

$$
    \frac{1}{n} \| Xw - y \|_{2}^{2} = \frac{1}{n} (Xw - y)^{\top} (Xw - y)
$$

(потому что $\| x \|_{2}^{2} = \langle x, x\rangle$)

Раскроем это выражение:

$$
\begin{align*}
    & (Xw - y)^{\top} (Xw - y) =\\
    &= (w^{\top} X^{\top} - y^{\top}) (Xw - y) =\\
    &= (w^{\top} X^{\top} X w - w^{\top} X^{\top} y) - (y^{\top} X w - y^{\top} y) =\\
    &= w^{\top} X^{\top} X w - 2 y^{\top} X w + y^{\top} y
\end{align*}
$$

Найдём градиент этой функции, т.е. все частные производные по весам (т.е. по $w_1, \ldots, w_d$).

$$
\begin{align*}
    &\frac{\partial}{\partial w} (w^{\top} X^{\top} X w - 2 y^{\top} X w + y^{\top} y) =\\
    &= 2 X^{\top} X w - 2 X^{\top} y = 0
\end{align*}
$$

Отсюда получаем итоговый ответ:

$$
\begin{align*}
    X^{\top} X w &= X^{\top} y\\
    w &= (X^{\top} X)^{-1} X^{\top} y
\end{align*}
$$

Полезная статья прорешение Линейной регрессии: https://habr.com/ru/post/474602/

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
X = np.linspace(-5, 5, 100)
y = 10 * X - 7


X_train = X[0::2].reshape(-1, 1)
y_train = y[0::2] + np.random.randn(50) * 10

X_test = X[1::2].reshape(-1, 1)
y_test = y[1::2] + np.random.randn(50) * 10

In [ ]:
X_train[1], y_train[1]

In [ ]:
def fit(X, y):
    n, k = X.shape
    X = np.hstack((X, np.ones((n, 1))))
    w = np.linalg.inv(X.T @ X) @ X.T @ y
    return w

def predict(X, w):
    n, k = X.shape
    X = np.hstack((X, np.ones((n, 1))))
    y_pred = X @ w
    return y_pred

weights = fit(X_train, y_train)
weights

In [ ]:
y_hat = predict(X_test, weights)

In [ ]:
plt.hist((y_test - y_hat)**2, bins = 20)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(X, y, label = 'data')
plt.scatter(X_train, y_train, label ='train')
plt.scatter(X_test, y_test, label ='test')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(X, y, label = 'data')
plt.scatter(X_train, y_train, label ='train')
plt.scatter(X_test, y_test, label ='test')
plt.plot(X[1::2], X[1::2].reshape(-1, 1).dot(weights[:-1]) + weights[-1], label = 'preds')
plt.legend()
plt.show()

**Определение (R2-score или коэффициент детерминации):**

$$ R^2 = 1 - \frac{MSE(y, \widehat{y})}{D y} $$

— это доля дисперсии зависимой переменной, объясняемая рассматриваемой моделью зависимости, то есть объясняющими переменными. Более точно — это единица минус доля необъяснённой дисперсии (дисперсии случайной ошибки модели, или условной по факторам дисперсии зависимой переменной) в дисперсии зависимой переменной. 

In [ ]:
from sklearn.metrics import r2_score

train_preds = predict(X_train, weights)
test_preds = predict(X_test, weights)

print('train r2', r2_score(y_train, train_preds))
print('test r2', r2_score(y_test, test_preds))

In [ ]:
from sklearn.metrics import mean_squared_error

train_preds = predict(X_train, weights)
test_preds = predict(X_test, weights)

print('train mse', mean_squared_error(y_train, train_preds))
print('test mse', mean_squared_error(y_test, test_preds))

### Ridge&Lasso

На практике точные формулы для подсчета коэффициентов линейной регрессии не используются, а используется метод градиентного спуска который состоит в подсчете производных от ошибки и шагу в направлении наискорейшего убывания функции (напомню, что мы стремимся минимизировать функцию потерь). Эти методы работают быстрее, чем точное вычисление обратных матриц и их перемножение.
Более того, во многих задачах это единственный способ обучить модель, так как не всегда (на самом деле почти никогда) удается выписать точную формулу для минимума сложного функционала ошибки.

Давайте рассмотрим реализации линейных регрессоров в библиотеке sklearn

Но сначала давайте поймём, зачем вообще нужна регуляризация. Рассмотрим проблему мультиколлинеарности. В упрощённом понимании, это означает, что признаки линейно зависимы. Посмотрим, к чему это ведёт.

In [ ]:
X_adversary = X_train.copy()
X_adversary[:, 0] = 2
print(X_train.shape, X_adversary.shape)

In [ ]:
w_adversary = fit(X_adversary, y_train)
w_adversary

**ВОПРОС** Почему так произошло??

In [ ]:
#TODO

**Что произошло**

Ранг матрицы $\mathrm{X_{adversary}}$ равен 1, а размерность признакового пространства равна 2. Из линейной алгебры известно, что ранг произведения матриц не превосходит минимального ранга этих двух матриц: 

$$
    \mathrm{rk} AB \leq \min \{\mathrm{rk} A, \mathrm{rk} B\}
$$

К чему это здесь приводит? Посмотрим на аналитическое решение
$$
    w = (X^{\top} X)^{-1} X^{\top} y
$$

Нас интересует компонента $(X^{\top} X)^{-1}$. Здесь обратная не определена, потому что ранг матрицы $X^{\top} X$ не превосходит единицы. При этом матрица $X^{\top} X$ -- квадратная, размера 2x2. Из линейной алгебры мы знаем, что квадратная матрица обратима только тогда, когда она полноранговая. Именно об этом Вам сигналит ошибка LinAlgError: Вы пытаетесь обратить матрицу, которую обращать нельзя. Что делать?

**Определение ($L_{p}$-регуляризация):**
Пусть задана линейная регрессия с вектором весов $w$ и функцией ошибок $\mathcal{L}(y, \widehat{y}(w))$ (например, среднеквадратичная ошибка), тогда к ней можно добавить $L_{p}$-регуляризацию изменив функционал ошибки следующим образом:

$$
    \mathcal{L}(y, \widehat{y}(w)) + \alpha \|w\|_{p}^{p} \to \min\limits_{w}
$$

Если $p$ равно 2, то это называют **Ridge**-регуляризацией, а если $p$ равно 1, то **Lasso**-регуляризацией.

**Утверждение:** $L_{2}$-регуляризация позволяет избежать этой проблемы ($L_{p}$-нормы для произвольных $p$, на самом деле, тоже, но это уже нетривиально доказывать).

**Определение (напоминание):** Собственные значения матрицы $A$ это такие числа $\lambda$, что существует ненулевой вектор $x$, такой что $Ax = \lambda x$

А как вообще понять, насколько плохо всё с матрицей в плане того, что с ней будет происходить при попытке её обратить?
Посчитать коэффициент обусловленности: отношение максимального собственного значения к минимальному. Чем он больше, тем всё хуже.

In [ ]:
eigenvals, eigenvectors = np.linalg.eig(X.reshape(1,-1).T @ X.reshape(1,-1))
eigenvals.max() / eigenvals.min()

## Решим задачу регрессии с библиотекой sklearn

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [ ]:
from sklearn.datasets import load_wine

In [ ]:
wine_data = load_wine()
wine_data

In [ ]:
X = pd.DataFrame(wine_data['data'], columns=wine_data['feature_names'])
y = wine_data['target']

In [ ]:
_ = X.hist(X.columns, figsize=(10, 10))

In [ ]:
import seaborn as sns

plt.figure(figsize = (10,6))
sns.heatmap(X.corr())

In [ ]:
sns.clustermap(X.corr())

In [ ]:
X.corr().loc['total_phenols', 'flavanoids']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=99, stratify=y
)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
regressor = LinearRegression()

regressor.fit(X_train, y_train)
test_predictions = regressor.predict(X_test)

print('test mse: ', mean_squared_error(y_test, test_predictions))
print('r2 score: ', r2_score(y_test, test_predictions))

In [ ]:
X

In [ ]:
plt.figure(figsize=(20, 8))
plt.bar(X.columns, regressor.coef_)

Теперь обратимся к методам с регуляризацией.

Ridge (L2-регуляризация) сильно штрафует за слишком большие веса и не очень за малые. При увеличении коэффициента перед регуляризатором веса меняются плавно

In [ ]:
alphas = np.linspace(1, 1000, 100)

weights = np.empty((len(X.columns), 0))
for alpha in alphas:
    ridge_regressor = Ridge(alpha)
    ridge_regressor.fit(X_train, y_train)
    weights = np.hstack((weights, ridge_regressor.coef_.reshape(-1, 1)))
plt.plot(alphas, weights.T)
plt.xlabel('regularization coef')
plt.ylabel('weight value')
plt.show()

Lasso (L1) одинаково сильно штрафует малые и большие веса, поэтому при достаточно большом коэффициенте регуляризации многие признаки становятся равными нулю, при этом остаются только наиболее инфромативные. Этот факт можно использовать для решения задачи отбора признаков.

In [ ]:
alphas = np.linspace(0.1, 1, 100)

plt.figure(figsize=(10, 5))
weights = np.empty((len(X.columns), 0))
for alpha in alphas:
    lasso_regressor = Lasso(alpha)
    lasso_regressor.fit(X_train, y_train)
    weights = np.hstack((weights, lasso_regressor.coef_.reshape(-1, 1)))
plt.plot(alphas, weights.T)
plt.xlabel('regularization coef')
plt.ylabel('weight value')
plt.grid()
plt.show()

In [ ]:
ridge = Ridge(0.1)
ridge.fit(X_train, y_train)
print('\n r2 score ridge: ', r2_score(y_test, ridge.predict(X_test)))
print('test mse ridge: ', mean_squared_error(y_test, ridge.predict(X_test)))

lasso = Lasso(0.1)
lasso.fit(X_train, y_train)
print('\n r2 score lasso: ', r2_score(y_test, lasso.predict(X_test)))
print('test mse lasso: ', mean_squared_error(y_test, lasso.predict(X_test)))